In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import zeus.notebook_utils.syspath as syspath
syspath.add_parent_folder()

In [ ]:
import random
from collections import defaultdict
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import rasterio
from kidney.datasets.kaggle import get_reader, SampleType
from zeus.utils import list_files
from zeus.plotting.utils import axes, calculate_layout

In [ ]:
reader = get_reader()

In [ ]:
train_keys = reader.get_keys(SampleType.Labeled)

In [ ]:
identity = rasterio.Affine(1, 0, 0, 0, 1, 0)

In [ ]:
for key in train_keys:
    meta = reader.fetch_meta(key)
    with rasterio.open(meta["tiff"], transform=identity) as dataset:
        height, width = shape = dataset.shape
        print(height, width, dataset.indexes)

In [ ]:
samples = defaultdict(dict)
for fn in list_files("/mnt/fast/data/kidney/images_32_1024"):
    image_type, image_id = Path(fn).stem.split(".")
    samples[image_id][image_type] = fn

In [ ]:
# x = samples["8242609fa_19584_10759_20608_11783"]
# img = np.asarray(PIL.Image.open(x["img"]))
# seg = np.asarray(PIL.Image.open(x["seg"]))
# plt.figure(figsize=(10,10))
# plt.imshow(img)
# plt.imshow(seg, alpha=0.3)
# plt.show()

In [ ]:
n = 7
keys = random.sample(samples.keys(), k=n*n)
canvas = axes(subplots=(n, n), figsize=(20, 20))
for key, ax in zip(keys, canvas.flat):
    x = samples[key]
    img = np.asarray(PIL.Image.open(x["img"]))
    seg = np.asarray(PIL.Image.open(x["seg"]))
    grayscale = img.ndim == 2
    ax.imshow(img, cmap="gray" if grayscale else None)
    ax.imshow(seg, alpha=0.3)
    ax.axis(False)
    ax.set_title("grayscale" if grayscale else "colored")